# Inteligencia Artificial, Machine Learning 2020-2021 - UIB

Por: Asier Miguel Alemany Ortiz

Repositorio: [https://github.com/Asmial/IAML2021](https://github.com/Asmial/IAML2021)

FIFA es una serie de videojuegos o simuladores de fútbol licenciados, publicados anualmente por Electronic Arts (EA). Durante los últimos 25 años han ido añadiendo características, que entre ellas destaca una mayor importancia en el tema de la simulación de un mánager, sobretodo en la parte de compra-venta de jugadores. Por eso, cada vez se dispone de una base de datos más extensa y detallada de cada uno de los jugadores.

Uno de los elementos más importantes es el valor económico de cada unos de los jugadores. Este valor se supone que está relacionado con las características de cada uno. Por eso, en limpiaremos la base de datos y generaremos un csv en el que junto con la librería scikit-learn, podamos predecir el valor del jugador a partir de sus características.

## Inicialización

Cargamos las librerías necesarias

In [1]:
import os

from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, mean_absolute_error, \
r2_score

import pandas as pd
import numpy as np
from sklearn.decomposition import PCA

import datetime

Cargamos el archivo CSV y le hacemos un análisis visual

In [2]:
pd.set_option('display.max_columns', None)
df = pd.read_csv(os.path.join('in', 'fifa.csv'))
df.head()

,Unnamed: 0,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,Club Logo,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Real Face,Position,Jersey Number,Joined,Loaned From,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,0,158023,L. Messi,31,https://cdn.sofifa.org/players/4/19/158023.png,Argentina,https://cdn.sofifa.org/flags/52.png,94,94,FC Barcelona,https://cdn.sofifa.org/teams/2/light/241.png,€110.5M,€565K,2202,Left,5.0,4.0,4.0,Medium/ Medium,Messi,Yes,RF,10.0,"Jul 1, 2004",NaN,2021,5'7,159lbs,88+2,88+2,88+2,92+2,93+2,93+2,93+2,92+2,93+2,93+2,93+2,91+2,84+2,84+2,84+2,91+2,64+2,61+2,61+2,61+2,64+2,59+2,47+2,47+2,47+2,59+2,84.0,95.0,70.0,90.0,86.0,97.0,93.0,94.0,87.0,96.0,91.0,86.0,91.0,95.0,95.0,85.0,68.0,72.0,59.0,94.0,48.0,22.0,94.0,94.0,75.0,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,€226.5M
1,1,20801,Cristiano Ronaldo,33,https://cdn.sofifa.org/players/4/19/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94,94,Juventus,https://cdn.sofifa.org/teams/2/light/45.png,€77M,€405K,2228,Right,5.0,4.0,5.0,High/ Low,C. Ronaldo,Yes,ST,7.0,"Jul 10, 2018",NaN,2022,6'2,183lbs,91+3,91+3,91+3,89+3,90+3,90+3,90+3,89+3,88+3,88+3,88+3,88+3,81+3,81+3,81+3,88+3,65+3,61+3,61+3,61+3,65+3,61+3,53+3,53+3,53+3,61+3,84.0,94.0,89.0,81.0,87.0,88.0,81.0,76.0,77.0,94.0,89.0,91.0,87.0,96.0,70.0,95.0,95.0,88.0,79.0,93.0,63.0,29.0,95.0,82.0,85.0,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,€127.1M
2,2,190871,Neymar Jr,26,https://cdn.sofifa.org/players/4/19/190871.png,Brazil,https://cdn.sofifa.org/flags/54.png,92,93,Paris Saint-Germain,https://cdn.sofifa.org/teams/2/light/73.png,€118.5M,€290K,2143,Right,5.0,5.0,5.0,High/ Medium,Neymar,Yes,LW,10.0,"Aug 3, 2017",NaN,2022,5'9,150lbs,84+3,84+3,84+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,88+3,81+3,81+3,81+3,88+3,65+3,60+3,60+3,60+3,65+3,60+3,47+3,47+3,47+3,60+3,79.0,87.0,62.0,84.0,84.0,96.0,88.0,87.0,78.0,95.0,94.0,90.0,96.0,94.0,84.0,80.0,61.0,81.0,49.0,82.0,56.0,36.0,89.0,87.0,81.0,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,€228.1M
3,3,193080,De Gea,27,https://cdn.sofifa.org/players/4/19/193080.png,Spain,https://cdn.sofifa.org/flags/45.png,91,93,Manchester United,https://cdn.sofifa.org/teams/2/light/11.png,€72M,€260K,1471,Right,4.0,3.0,1.0,Medium/ Medium,Lean,Yes,GK,1.0,"Jul 1, 2011",NaN,2020,6'4,168lbs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,13.0,21.0,50.0,13.0,18.0,21.0,19.0,51.0,42.0,57.0,58.0,60.0,90.0,43.0,31.0,67.0,43.0,64.0,12.0,38.0,30.0,12.0,68.0,40.0,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,€138.6M
4,4,192985,K. De Bruyne,27,https://cdn.sofifa.org/players/4/19/192985.png,Belgium,https://cdn.sofifa.org/flags/7.png,91,92,Manchester City,https://cdn.sofifa.org/teams/2/light/10.png,€102M,€355K,2281,Right,4.0,5.0,4.0,High/ High,Normal,Yes,RCM,7.0,"Aug 30, 2015",NaN,2023,5'11,154lbs,82+3,82+3,82+3,87+3,87+3,87+3,87+3,87+3,88+3,88+3,88+3,88+3,87+3,87+3,87+3,88+3,77+3,77+3,77+3,77+3,77+3,73+3,66+3,66+3,66+3,73+3,93.0,82.0,55.0,92.0,82.0,86.0,85.0,83.0,91.0,91.0,78.0,76.0,79.0,91.0,77.0,91.0,63.0,90.0,75.0,91.0,76.0,61.0,87.0,94.0,79.0,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,€196.4M


Como podemos ver hay campos que ya de por sí son inútiles porque no aportan nada de información relevante, procedemos a eliminarlos en la siguienete sección.

## Eliminación de columnas innecesarias

Como los URLs son sencillamente archivos de imágen que llevan a una página 404, no nos isrven para analizarlos

In [3]:
df = df.drop(['Photo','Flag','Club Logo'], axis=1)

Como el id y Unnamed: 0 son sencillamente variables para indexar al jugador, las eliminamos tambíen

In [4]:
df = df.iloc[:, 2:]

El nombre y el número de la camiseta son técnicamente irrelevantes excepto para el número 1, que es el portero principal, pero como ya tenemos la posición en la columna Position, vamos a comprobar si hay alguien que tenga el 1 y no sea 'GK' (Goal Keeper).

In [5]:
len(df.loc[(df['Jersey Number'] == 1) & (df['Position'] != 'GK')])

0

Como podemos ver, ambas son variables personales, así que las eliminamos

In [6]:
df = df.drop(['Name','Jersey Number'], axis = 1)

## Estadísticas

Eliminamos las filas de las estadísticas nulas

In [7]:
df = df.dropna(subset= ['Crossing'])

Comprobamos si el numero de la izquierda del '+' es dependiente de el de la derecha

In [8]:
posattr = ['LS','ST','RS','LW','LF','CF','RF','RW','LAM','CAM','RAM', \
           'LM','LCM','CM','RCM','RM','LWB','LDM','CDM','RDM','RWB', \
           'LB','LCB','CB','RCB','RB']

wplus = 0
woplus = 0
for i in posattr:
    wplus += len(df['ST'].str[0:-2].unique())
    woplus += len(df['ST'].unique())
print(wplus)
print(woplus)

1560
2444


No parece que lo sean, por eso mismo los trataremos distinto.

Suponemos que lo de la derecha del + es el potencial máximo, por eso, haremos la media entre la base y la base más el máximo, ya que un jugador con 50+3 es peor que uno con 52+1 debido a que la base inicial es superior.

In [9]:
dfc = df.copy()

def plusclean(x):
    y = x.split('+')
    return int(y[0]) + int(y[1])/2

for i in posattr:
    dfc[i] = dfc[i].loc[dfc[i].notna()].apply(lambda x: plusclean(x))

df = dfc

In [10]:
df

,Age,Nationality,Overall,Potential,Club,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Real Face,Position,Joined,Loaned From,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,31,Argentina,94,94,FC Barcelona,€110.5M,€565K,2202,Left,5.0,4.0,4.0,Medium/ Medium,Messi,Yes,RF,"Jul 1, 2004",NaN,2021,5'7,159lbs,89.0,89.0,89.0,93.0,94.0,94.0,94.0,93.0,94.0,94.0,94.0,92.0,85.0,85.0,85.0,92.0,65.0,62.0,62.0,62.0,65.0,60.0,48.0,48.0,48.0,60.0,84.0,95.0,70.0,90.0,86.0,97.0,93.0,94.0,87.0,96.0,91.0,86.0,91.0,95.0,95.0,85.0,68.0,72.0,59.0,94.0,48.0,22.0,94.0,94.0,75.0,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,€226.5M
1,33,Portugal,94,94,Juventus,€77M,€405K,2228,Right,5.0,4.0,5.0,High/ Low,C. Ronaldo,Yes,ST,"Jul 10, 2018",NaN,2022,6'2,183lbs,92.5,92.5,92.5,90.5,91.5,91.5,91.5,90.5,89.5,89.5,89.5,89.5,82.5,82.5,82.5,89.5,66.5,62.5,62.5,62.5,66.5,62.5,54.5,54.5,54.5,62.5,84.0,94.0,89.0,81.0,87.0,88.0,81.0,76.0,77.0,94.0,89.0,91.0,87.0,96.0,70.0,95.0,95.0,88.0,79.0,93.0,63.0,29.0,95.0,82.0,85.0,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,€127.1M
2,26,Brazil,92,93,Paris Saint-Germain,€118.5M,€290K,2143,Right,5.0,5.0,5.0,High/ Medium,Neymar,Yes,LW,"Aug 3, 2017",NaN,2022,5'9,150lbs,85.5,85.5,85.5,90.5,90.5,90.5,90.5,90.5,90.5,90.5,90.5,89.5,82.5,82.5,82.5,89.5,66.5,61.5,61.5,61.5,66.5,61.5,48.5,48.5,48.5,61.5,79.0,87.0,62.0,84.0,84.0,96.0,88.0,87.0,78.0,95.0,94.0,90.0,96.0,94.0,84.0,80.0,61.0,81.0,49.0,82.0,56.0,36.0,89.0,87.0,81.0,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,€228.1M
3,27,Spain,91,93,Manchester United,€72M,€260K,1471,Right,4.0,3.0,1.0,Medium/ Medium,Lean,Yes,GK,"Jul 1, 2011",NaN,2020,6'4,168lbs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,13.0,21.0,50.0,13.0,18.0,21.0,19.0,51.0,42.0,57.0,58.0,60.0,90.0,43.0,31.0,67.0,43.0,64.0,12.0,38.0,30.0,12.0,68.0,40.0,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,€138.6M
4,27,Belgium,91,92,Manchester City,€102M,€355K,2281,Right,4.0,5.0,4.0,High/ High,Normal,Yes,RCM,"Aug 30, 2015",NaN,2023,5'11,154lbs,83.5,83.5,83.5,88.5,88.5,88.5,88.5,88.5,89.5,89.5,89.5,89.5,88.5,88.5,88.5,89.5,78.5,78.5,78.5,78.5,78.5,74.5,67.5,67.5,67.5,74.5,93.0,82.0,55.0,92.0,82.0,86.0,85.0,83.0,91.0,91.0,78.0,76.0,79.0,91.0,77.0,91.0,63.0,90.0,75.0,91.0,76.0,61.0,87.0,94.0,79.0,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,€196.4M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18202,19,England,47,65,Crewe Alexandra,€60K,€1K,1307,Right,1.0,2.0,2.0,Medium/ Medium,Lean,No,CM,"May 3, 2017",NaN,2019,5'9,134lbs,43.0,43.0,43.0,45.0,45.0,45.0,45.0,45.0,46.0,46.0,46.0,45.0,46.0,46.0,46.0,45.0,45.0,46.0,46.0,46.0,45.0,46.0,46.0,46.0,46.0,46.0,34.0,38.0,40.0,49.0,25.0,42.0,30.0,34.0,45.0,43.0,54.0,57.0,60.0,49.0,76.0,43.0,55.0,40.0,47.0,38.0,46.0,46.0,39.0,52.0,43.0,45.0,40.0,48.0,47.0,10.0,13.0,7.0,8.0,9.0,€143K
18203,19,Sweden,47,63,Trelleborgs FF,€60K,€1K,1098,Right,1.0,2.0,2.0,Medium/ Medium,Normal,No,ST,"Mar 19, 2018",NaN,2020,6'3,170lbs,46.0,46.0,46.0,40.0,43.0,43.0,43.0,40.0,41.0,41.0,41.0,39.0,36.0,36.0,36.0,39.0,31.0,32.0,32.0,32.0,31.0,30.0,33.0,33.0,33.0,30.0,23.0,52.0,52.0,43.0,36.0,39.0,32.0,20.0,25.0,40.0,41.0,39.0,38.0,40.0,52.0,41.0,47.0,43.0,67.0,42.0,47.0,16.0,46.0,33.0,43.0,42.0,

Como podemos ver arriba, el portero De Gea tiene nan en todas las posiciones que no son GK, por eso miraremos si hay mas porteros que tienen ese problema

In [11]:
df.loc[(df['Position'] == 'GK')]

,Age,Nationality,Overall,Potential,Club,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Real Face,Position,Joined,Loaned From,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
3,27,Spain,91,93,Manchester United,€72M,€260K,1471,Right,4.0,3.0,1.0,Medium/ Medium,Lean,Yes,GK,"Jul 1, 2011",NaN,2020,6'4,168lbs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,13.0,21.0,50.0,13.0,18.0,21.0,19.0,51.0,42.0,57.0,58.0,60.0,90.0,43.0,31.0,67.0,43.0,64.0,12.0,38.0,30.0,12.0,68.0,40.0,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,€138.6M
9,25,Slovenia,90,93,Atlético Madrid,€68M,€94K,1331,Right,3.0,3.0,1.0,Medium/ Medium,Normal,Yes,GK,"Jul 16, 2014",NaN,2021,6'2,192lbs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0,11.0,15.0,29.0,13.0,12.0,13.0,14.0,26.0,16.0,43.0,60.0,67.0,86.0,49.0,22.0,76.0,41.0,78.0,12.0,34.0,19.0,11.0,70.0,11.0,70.0,27.0,12.0,18.0,86.0,92.0,78.0,88.0,89.0,€144.5M
18,26,Germany,89,92,FC Barcelona,€58M,€240K,1328,Right,3.0,4.0,1.0,Medium/ Medium,Normal,Yes,GK,"Jul 1, 2014",NaN,2022,6'2,187lbs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.0,14.0,11.0,36.0,14.0,17.0,18.0,12.0,42.0,18.0,38.0,50.0,37.0,85.0,43.0,22.0,79.0,35.0,79.0,10.0,43.0,22.0,11.0,69.0,25.0,69.0,25.0,13.0,10.0,87.0,85.0,88.0,85.0,90.0,€123.3M
19,26,Belgium,89,90,Real Madrid,€53.5M,€240K,1311,Left,4.0,2.0,1.0,Medium/ Medium,Courtois,Yes,GK,"Aug 9, 2018",NaN,2024,6'6,212lbs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.0,14.0,13.0,33.0,12.0,13.0,19.0,20.0,35.0,23.0,46.0,52.0,61.0,84.0,45.0,36.0,68.0,38.0,70.0,17.0,23.0,15.0,13.0,44.0,27.0,66.0,20.0,18.0,16.0,85.0,91.0,72.0,86.0,88.0,€113.7M
22,32,Germany,89,89,FC Bayern München,€38M,€130K,1473,Right,5.0,4.0,1.0,Medium/ Medium,Normal,Yes,GK,"Jul 1, 2011",NaN,2021,6'4,203lbs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.0,13.0,25.0,55.0,11.0,30.0,14.0,11.0,59.0,48.0,54.0,60.0,51.0,84.0,35.0,25.0,77.0,43.0,80.0,16.0,29.0,30.0,12.0,70.0,47.0,70.0,17.0,10.0,11.0,90.0,86.0,91.0,87.0,87.0,€62.7M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18178,18,Sweden,48,65,Dalkurd FF,€50K,€1K,738,Right,1.0,2.0,1.0,Medium/ Medium,Lean,No,GK,"Apr 4, 2018",NaN,2018,6'0,176lbs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,7.0,11.0,22.0,6.0,8.0,10.0,8.0,20.0,12.0,24.0,28.0,33.0,40.0,29.0,15.0,48.0,20.0,38.0,6.0,20.0,10.0,7.0,18.0,12.0,28.0,16.0,11.0,10.0,47.0,46.0,50.0,45.0,51.0,€94K
18180,22,Scotland,48,58,St. Johnstone FC,€40K,€1K,987,Right,1.0,2.0,1.0,Medium/ Medium,Lean,No,GK,"Jul 1, 2013",NaN,2019,6'1,172lbs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.0,15.0,15.0,26.0,14.0,13.0,12.0,19.0,19.0,25.0,40.0,42.0,45.0,47.0,52.0,25.0,55.0,33.0,58.0,20.0,20.0,29.0,13.0,20.0,29.0,28.0,12.0,15.0,16.0,45.0,49.0,50.0,50.0,45.0,€78K
18183,44,England,48,48,Cambridge United,€0,€1K,774,Right,1.0,2.0,1.0,Medium/ Medium,Normal,No,GK,"Aug 17, 2018",N

La idea es obtener la media de sus compañeros de equipo, el problema es que hay equipos vacíos

In [12]:
print(len(df.loc[(df['Club'].isna())]))
print(len(df.loc[(df['Club'].isna()) & (df['Value'] == '€0')]))

241
241


Por suerte, todos los jugadores sin equipo no valen nada, así que eliminamos los datos

In [13]:
df = df.dropna(subset=['Club'])

Para el resto de porteros asociamos los datos de posición de la media de sus compañeros de equipo (¡¡¡tarda mucho!!!)

In [14]:
dfc = df.copy()
for club in dfc.loc[dfc['Position'] == 'GK']['Club'].unique():
    for pos in posattr:
        dfc.loc[(dfc['Club'] == club) & (dfc['Position'] == 'GK'), \
                pos] = \
        dfc.loc[(dfc['Club'] == club) & (dfc['Position'] != 'GK'), \
                pos].mean()
df = dfc

Comprobamos los resultados

In [15]:
dfc.loc[dfc['Position'] == 'GK']

,Age,Nationality,Overall,Potential,Club,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Real Face,Position,Joined,Loaned From,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
3,27,Spain,91,93,Manchester United,€72M,€260K,1471,Right,4.0,3.0,1.0,Medium/ Medium,Lean,Yes,GK,"Jul 1, 2011",NaN,2020,6'4,168lbs,69.266667,69.266667,69.266667,70.566667,70.466667,70.466667,70.466667,70.566667,70.966667,70.966667,70.966667,71.266667,70.466667,70.466667,70.466667,71.266667,68.733333,68.733333,68.733333,68.733333,68.733333,67.733333,66.633333,66.633333,66.633333,67.733333,17.0,13.0,21.0,50.0,13.0,18.0,21.0,19.0,51.0,42.0,57.0,58.0,60.0,90.0,43.0,31.0,67.0,43.0,64.0,12.0,38.0,30.0,12.0,68.0,40.0,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,€138.6M
9,25,Slovenia,90,93,Atlético Madrid,€68M,€94K,1331,Right,3.0,3.0,1.0,Medium/ Medium,Normal,Yes,GK,"Jul 16, 2014",NaN,2021,6'2,192lbs,65.966667,65.966667,65.966667,67.466667,67.333333,67.333333,67.333333,67.466667,67.333333,67.333333,67.333333,68.133333,66.333333,66.333333,66.333333,68.133333,65.033333,64.133333,64.133333,64.133333,65.033333,64.200000,62.466667,62.466667,62.466667,64.200000,13.0,11.0,15.0,29.0,13.0,12.0,13.0,14.0,26.0,16.0,43.0,60.0,67.0,86.0,49.0,22.0,76.0,41.0,78.0,12.0,34.0,19.0,11.0,70.0,11.0,70.0,27.0,12.0,18.0,86.0,92.0,78.0,88.0,89.0,€144.5M
18,26,Germany,89,92,FC Barcelona,€58M,€240K,1328,Right,3.0,4.0,1.0,Medium/ Medium,Normal,Yes,GK,"Jul 1, 2014",NaN,2022,6'2,187lbs,68.948276,68.948276,68.948276,71.810345,71.844828,71.844828,71.844828,71.810345,72.844828,72.844828,72.844828,72.500000,72.706897,72.706897,72.706897,72.500000,69.603448,69.775862,69.775862,69.775862,69.603448,68.396552,65.879310,65.879310,65.879310,68.396552,15.0,14.0,11.0,36.0,14.0,17.0,18.0,12.0,42.0,18.0,38.0,50.0,37.0,85.0,43.0,22.0,79.0,35.0,79.0,10.0,43.0,22.0,11.0,69.0,25.0,69.0,25.0,13.0,10.0,87.0,85.0,88.0,85.0,90.0,€123.3M
19,26,Belgium,89,90,Real Madrid,€53.5M,€240K,1311,Left,4.0,2.0,1.0,Medium/ Medium,Courtois,Yes,GK,"Aug 9, 2018",NaN,2024,6'6,212lbs,67.948276,67.948276,67.948276,70.465517,70.258621,70.258621,70.258621,70.465517,70.948276,70.948276,70.948276,71.224138,70.568966,70.568966,70.568966,71.224138,68.741379,68.396552,68.396552,68.396552,68.741379,67.775862,65.224138,65.224138,65.224138,67.775862,14.0,14.0,13.0,33.0,12.0,13.0,19.0,20.0,35.0,23.0,46.0,52.0,61.0,84.0,45.0,36.0,68.0,38.0,70.0,17.0,23.0,15.0,13.0,44.0,27.0,66.0,20.0,18.0,16.0,85.0,91.0,72.0,86.0,88.0,€113.7M
22,32,Germany,89,89,FC Bayern München,€38M,€130K,1473,Right,5.0,4.0,1.0,Medium/ Medium,Normal,Yes,GK,"Jul 1, 2011",NaN,2021,6'4,203lbs,68.660000,68.660000,68.660000,70.300000,70.260000,70.260000,70.260000,70.300000,70.940000,70.940000,70.940000,70.900000,70.420000,70.420000,70.420000,70.900000,67.220000,67.180000,67.180000,67.180000,67.220000,66.140000,64.100000,64.100000,64.100000,66.140000,15.0,13.0,25.0,55.0,11.0,30.0,14.0,11.0,59.0,48.0,54.0,60.0,51.0,84.0,35.0,25.0,77.0,43.0,80.0,16.0,29.0,30.0,12.0,70.0,47.0,70.0,17.0,10.0,11.0,90.0,86.0,91.0,87.0,87.0,€62.7M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18178,18,Sweden,48,65,Dalkurd FF,€50K,€1K,738,Right,1.0,2.0,1.0,Medium/ Medium,Lean,No,GK,"Apr 4, 2018",NaN,2018,6'0,176lbs,51.526316,51.5263

## Precios

En la tabla hay varios valores numéricos indicando el precio, vamos a analizar sus prefijos y sufijos

In [16]:
pricescols = ['Value', 'Wage', 'Release Clause']
for i in pricescols:
    for j in [0,-1]:
        print(pd.Series(df[i].str.strip().str[j]).unique())

['€']
['M' '0' 'K']
['€']
['K']
['€' nan]
['M' nan 'K']


Posibles monedas: €

Posibles terminadores: K, M

Los convertimos a floats según los prefijos y sufijos:

In [17]:
newnames = ['value', 'wage', 'rclause']
dfc = df.copy()

def fix(x):
    print(x)
    return float(x[1:-1]) * 1000000

for count, col in enumerate(pricescols):
    dfc[newnames[count]] = df[col].apply(lambda x:  \
            x if isinstance(x, float)
            else float(x[1:-1]) * 1000 if x[-1] == 'K' \
            else float(x[1:-1]) * 1000000 if x[-1] == 'M' \
            else float(x[1:len(x)]))
df = dfc
df

,Age,Nationality,Overall,Potential,Club,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Real Face,Position,Joined,Loaned From,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause,value,wage,rclause
0,31,Argentina,94,94,FC Barcelona,€110.5M,€565K,2202,Left,5.0,4.0,4.0,Medium/ Medium,Messi,Yes,RF,"Jul 1, 2004",NaN,2021,5'7,159lbs,89.000000,89.000000,89.000000,93.000000,94.000000,94.000000,94.000000,93.000000,94.000000,94.000000,94.000000,92.000000,85.000000,85.000000,85.000000,92.000000,65.000000,62.000000,62.000000,62.000000,65.000000,60.000000,48.000000,48.000000,48.000000,60.000000,84.0,95.0,70.0,90.0,86.0,97.0,93.0,94.0,87.0,96.0,91.0,86.0,91.0,95.0,95.0,85.0,68.0,72.0,59.0,94.0,48.0,22.0,94.0,94.0,75.0,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,€226.5M,110500000.0,565000.0,226500000.0
1,33,Portugal,94,94,Juventus,€77M,€405K,2228,Right,5.0,4.0,5.0,High/ Low,C. Ronaldo,Yes,ST,"Jul 10, 2018",NaN,2022,6'2,183lbs,92.500000,92.500000,92.500000,90.500000,91.500000,91.500000,91.500000,90.500000,89.500000,89.500000,89.500000,89.500000,82.500000,82.500000,82.500000,89.500000,66.500000,62.500000,62.500000,62.500000,66.500000,62.500000,54.500000,54.500000,54.500000,62.500000,84.0,94.0,89.0,81.0,87.0,88.0,81.0,76.0,77.0,94.0,89.0,91.0,87.0,96.0,70.0,95.0,95.0,88.0,79.0,93.0,63.0,29.0,95.0,82.0,85.0,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,€127.1M,77000000.0,405000.0,127100000.0
2,26,Brazil,92,93,Paris Saint-Germain,€118.5M,€290K,2143,Right,5.0,5.0,5.0,High/ Medium,Neymar,Yes,LW,"Aug 3, 2017",NaN,2022,5'9,150lbs,85.500000,85.500000,85.500000,90.500000,90.500000,90.500000,90.500000,90.500000,90.500000,90.500000,90.500000,89.500000,82.500000,82.500000,82.500000,89.500000,66.500000,61.500000,61.500000,61.500000,66.500000,61.500000,48.500000,48.500000,48.500000,61.500000,79.0,87.0,62.0,84.0,84.0,96.0,88.0,87.0,78.0,95.0,94.0,90.0,96.0,94.0,84.0,80.0,61.0,81.0,49.0,82.0,56.0,36.0,89.0,87.0,81.0,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,€228.1M,118500000.0,290000.0,228100000.0
3,27,Spain,91,93,Manchester United,€72M,€260K,1471,Right,4.0,3.0,1.0,Medium/ Medium,Lean,Yes,GK,"Jul 1, 2011",NaN,2020,6'4,168lbs,69.266667,69.266667,69.266667,70.566667,70.466667,70.466667,70.466667,70.566667,70.966667,70.966667,70.966667,71.266667,70.466667,70.466667,70.466667,71.266667,68.733333,68.733333,68.733333,68.733333,68.733333,67.733333,66.633333,66.633333,66.633333,67.733333,17.0,13.0,21.0,50.0,13.0,18.0,21.0,19.0,51.0,42.0,57.0,58.0,60.0,90.0,43.0,31.0,67.0,43.0,64.0,12.0,38.0,30.0,12.0,68.0,40.0,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,€138.6M,72000000.0,260000.0,138600000.0
4,27,Belgium,91,92,Manchester City,€102M,€355K,2281,Right,4.0,5.0,4.0,High/ High,Normal,Yes,RCM,"Aug 30, 2015",NaN,2023,5'11,154lbs,83.500000,83.500000,83.500000,88.500000,88.500000,88.500000,88.500000,88.500000,89.500000,89.500000,89.500000,89.500000,88.500000,88.500000,88.500000,89.500000,78.500000,78.500000,78.500000,78.500000,78.500000,74.500000,67.500000,67.500000,67.500000,74.500000,93.0,82.0,55.0,92.0,82.0,86.0,85.0,83.0,91.0,91.0,78.0,76.0,79.0,91.0,77.0,91.0,63.0,90.0,75.0,91.0,76.0,61.0,87.0,94.0,79.0,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,€196.4M,102000000.0,355000.0,196400000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

Veamos cuantos precios, sueldos y valores de los jugadores que están vacíos

In [18]:
for i in pricescols:
    print(len(df.loc[df[i].isna()]))

0
0
1275


Podemos que hay muchos, como hemos tirado a los jugadores sin club, vamos a aplicar la misma lógica que a los porteros, que  es aplicarles la media de su club

In [19]:
rclausecopy = df['rclause'].copy()
for i in df.loc[dfc['Release Clause'].isna()].index:
    club = df['Club'].loc[[i]].item()
    newvalue = df.loc[(df['Club'] == club) & \
                      (df['rclause'].isna() == False)]['rclause'].mean()
    rclausecopy.loc[[i]] = newvalue
df['rclause'] = rclausecopy
print(len(df.loc[df['rclause'].isna()]))

0


Como podemos ver ya no hay ninguno vacío, entonces:

Eliminamos las columnas anteriores de los precios en forma de string

In [20]:
df = df.drop(pricescols, axis=1)

In [21]:
df

,Age,Nationality,Overall,Potential,Club,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Real Face,Position,Joined,Loaned From,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,value,wage,rclause
0,31,Argentina,94,94,FC Barcelona,2202,Left,5.0,4.0,4.0,Medium/ Medium,Messi,Yes,RF,"Jul 1, 2004",NaN,2021,5'7,159lbs,89.000000,89.000000,89.000000,93.000000,94.000000,94.000000,94.000000,93.000000,94.000000,94.000000,94.000000,92.000000,85.000000,85.000000,85.000000,92.000000,65.000000,62.000000,62.000000,62.000000,65.000000,60.000000,48.000000,48.000000,48.000000,60.000000,84.0,95.0,70.0,90.0,86.0,97.0,93.0,94.0,87.0,96.0,91.0,86.0,91.0,95.0,95.0,85.0,68.0,72.0,59.0,94.0,48.0,22.0,94.0,94.0,75.0,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,110500000.0,565000.0,226500000.0
1,33,Portugal,94,94,Juventus,2228,Right,5.0,4.0,5.0,High/ Low,C. Ronaldo,Yes,ST,"Jul 10, 2018",NaN,2022,6'2,183lbs,92.500000,92.500000,92.500000,90.500000,91.500000,91.500000,91.500000,90.500000,89.500000,89.500000,89.500000,89.500000,82.500000,82.500000,82.500000,89.500000,66.500000,62.500000,62.500000,62.500000,66.500000,62.500000,54.500000,54.500000,54.500000,62.500000,84.0,94.0,89.0,81.0,87.0,88.0,81.0,76.0,77.0,94.0,89.0,91.0,87.0,96.0,70.0,95.0,95.0,88.0,79.0,93.0,63.0,29.0,95.0,82.0,85.0,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,77000000.0,405000.0,127100000.0
2,26,Brazil,92,93,Paris Saint-Germain,2143,Right,5.0,5.0,5.0,High/ Medium,Neymar,Yes,LW,"Aug 3, 2017",NaN,2022,5'9,150lbs,85.500000,85.500000,85.500000,90.500000,90.500000,90.500000,90.500000,90.500000,90.500000,90.500000,90.500000,89.500000,82.500000,82.500000,82.500000,89.500000,66.500000,61.500000,61.500000,61.500000,66.500000,61.500000,48.500000,48.500000,48.500000,61.500000,79.0,87.0,62.0,84.0,84.0,96.0,88.0,87.0,78.0,95.0,94.0,90.0,96.0,94.0,84.0,80.0,61.0,81.0,49.0,82.0,56.0,36.0,89.0,87.0,81.0,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,118500000.0,290000.0,228100000.0
3,27,Spain,91,93,Manchester United,1471,Right,4.0,3.0,1.0,Medium/ Medium,Lean,Yes,GK,"Jul 1, 2011",NaN,2020,6'4,168lbs,69.266667,69.266667,69.266667,70.566667,70.466667,70.466667,70.466667,70.566667,70.966667,70.966667,70.966667,71.266667,70.466667,70.466667,70.466667,71.266667,68.733333,68.733333,68.733333,68.733333,68.733333,67.733333,66.633333,66.633333,66.633333,67.733333,17.0,13.0,21.0,50.0,13.0,18.0,21.0,19.0,51.0,42.0,57.0,58.0,60.0,90.0,43.0,31.0,67.0,43.0,64.0,12.0,38.0,30.0,12.0,68.0,40.0,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,72000000.0,260000.0,138600000.0
4,27,Belgium,91,92,Manchester City,2281,Right,4.0,5.0,4.0,High/ High,Normal,Yes,RCM,"Aug 30, 2015",NaN,2023,5'11,154lbs,83.500000,83.500000,83.500000,88.500000,88.500000,88.500000,88.500000,88.500000,89.500000,89.500000,89.500000,89.500000,88.500000,88.500000,88.500000,89.500000,78.500000,78.500000,78.500000,78.500000,78.500000,74.500000,67.500000,67.500000,67.500000,74.500000,93.0,82.0,55.0,92.0,82.0,86.0,85.0,83.0,91.0,91.0,78.0,76.0,79.0,91.0,77.0,91.0,63.0,90.0,75.0,91.0,76.0,61.0,87.0,94.0,79.0,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,102000000.0,355000.0,196400000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18202,19,England,47,65,Crewe Alexandra,1307,Right,1.0,2.0,2.0,Medium/ Medium,Lean,No,CM,"May 3, 

## Atributos del personaje

Estos atributos són meramente atributos que son como se refleja el personaje en el juego y deberían de tener poca influencia en el juego, igulamente, quizás estas proporciones hagan que el jugador esté más cotizado, por eso, los trataremos. También trataremos cosas como si el personaje tiene un modelo único en el juego, tanto de cara como de foto, ya que la fama del personaje puede tener una influencia directa sobre el precio.

El peso está en libras, quitamos el sufijo y lo convertimos a int

In [22]:
dfc = df.copy()
dfc['weight'] = dfc['Weight'].apply(lambda x: int(x[0:-3]))
dfc = dfc.drop(['Weight'], axis=1)
df = dfc

La altura está en el formato inglés, como no es un formato en base 10, lo convertimos a centímetros

In [23]:
def tocm(x):
    y = x.split("'")
    return int(y[0]) * 30.48 + int(y[1]) * 2.54

dfc = df.copy()
dfc['height'] = dfc['Height'].apply(lambda x: tocm(x))
dfc = dfc.drop(['Height'], axis=1)
df = dfc

El pie prefierido del jugador es importante para nuestra tabla, como solo puede ser o derecho o izquierdo, ponemos un booleano que si es 1, prefiere usar el pie derecho.

In [24]:
df = df.dropna(subset = ['Preferred Foot'])

footsplit = df.pop('Preferred Foot')
footsplit.name = 'rfoot'
footsplit.loc[footsplit == 'Right'] = 1
footsplit.loc[footsplit == 'Left'] = 0

df = pd.concat([df.reset_index(drop=True), \
                footsplit.reset_index(drop=True)], axis=1, sort=False)

Como se ha dicho antes, que tenga una cara modelada es importante, así que haremos lo mismo que con el pie derecho

In [25]:
df = df.dropna(subset = ['Real Face'])

rf = df.pop('Real Face')
rf.name = 'rf'
rf.loc[rf == 'Yes'] = 1
rf.loc[rf == 'No'] = 0

df = pd.concat([df.reset_index(drop=True), \
                rf.reset_index(drop=True)], axis=1, sort=False)

Comprobamos los tipos de cuerpos:

In [26]:
df['Body Type'].value_counts()

Normal                 10436
Lean                    6351
Stocky                  1124
C. Ronaldo                 1
Akinfenwa                  1
Courtois                   1
Neymar                     1
Shaqiri                    1
PLAYER_BODY_TYPE_25        1
Messi                      1
Name: Body Type, dtype: int64

Como podemos ver hay cuerpos que su atributo es único, uno de ellos llama la atencion porque no es ningún nombre, lo buscamos a ver quién es:

In [27]:
df.loc[df['Body Type'] == 'PLAYER_BODY_TYPE_25']

,Age,Nationality,Overall,Potential,Club,Special,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Position,Joined,Loaned From,Contract Valid Until,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,value,wage,rclause,weight,height,rfoot,rf
26,26,Egypt,88,89,Liverpool,2146,3.0,3.0,4.0,High/ Medium,PLAYER_BODY_TYPE_25,RM,"Jul 1, 2017",NaN,2023,84.5,84.5,84.5,88.5,87.5,87.5,87.5,88.5,87.5,87.5,87.5,87.5,81.5,81.5,81.5,87.5,71.5,67.5,67.5,67.5,71.5,67.5,58.5,58.5,58.5,67.5,78.0,90.0,59.0,82.0,73.0,89.0,83.0,60.0,72.0,88.0,94.0,91.0,91.0,91.0,88.0,77.0,68.0,84.0,70.0,83.0,63.0,55.0,90.0,82.0,61.0,91.0,38.0,43.0,41.0,14.0,14.0,9.0,11.0,14.0,69500000.0,255000.0,137300000.0,157,175.26,0,1


Mohammed Salah fue uno de los nominados a balón de oro de 2019, eso significa que seguramente su cuerpo si que sea único por su fama. Por eso, a todos los jugadores con un cuerpo único les pondremos que su cuerpo es 'unique':

In [28]:
unique_bodies = ['Akinfenwa','PLAYER_BODY_TYPE_25','Messi','Neymar', \
                 'C. Ronaldo','Shaqiri','Courtois']
btype = df['Body Type'].copy()
for i in unique_bodies:
    btype.loc[btype == i ] = 'unique'
df = df.drop(['Body Type'], axis = 1)
df = pd.concat([df.reset_index(drop=True), btype.reset_index(drop=True)], \
               axis=1, sort=False)

Analizamos el work rate:

In [29]:
df['Work Rate'].unique()

array(['Medium/ Medium', 'High/ Low', 'High/ Medium', 'High/ High',
       'Medium/ High', 'Medium/ Low', 'Low/ High', 'Low/ Medium',
       'Low/ Low'], dtype=object)

El work rate lo dividiremos en alto y bajo (ariba y debajo del '/')

Factorizamos de forma que:
* Low = 0
* Medium = 1
* High = 2

Lo hacemos así porque son categóricos al fin y al cabo (Low < Medium < High)

In [30]:
df = df.dropna(subset = ['Work Rate'])
wrsplit = df.pop('Work Rate').str.split('/ ').array
wrdf = pd.DataFrame(wrsplit, columns = ['wrh','wrl'])

for i in wrdf.columns:
    wrdf[i].loc[wrdf[i] == 'High'] = 2
    wrdf[i].loc[wrdf[i] == 'Medium'] = 1
    wrdf[i].loc[wrdf[i] == 'Low'] = 0

df = pd.concat([df.reset_index(drop=True),wrdf.reset_index(drop=True)], \
               axis=1, sort=False)

In [31]:
df

,Age,Nationality,Overall,Potential,Club,Special,International Reputation,Weak Foot,Skill Moves,Position,Joined,Loaned From,Contract Valid Until,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,value,wage,rclause,weight,height,rfoot,rf,Body Type,wrh,wrl
0,31,Argentina,94,94,FC Barcelona,2202,5.0,4.0,4.0,RF,"Jul 1, 2004",NaN,2021,89.000000,89.000000,89.000000,93.000000,94.000000,94.000000,94.000000,93.000000,94.000000,94.000000,94.000000,92.000000,85.000000,85.000000,85.000000,92.000000,65.000000,62.000000,62.000000,62.000000,65.000000,60.000000,48.000000,48.000000,48.000000,60.000000,84.0,95.0,70.0,90.0,86.0,97.0,93.0,94.0,87.0,96.0,91.0,86.0,91.0,95.0,95.0,85.0,68.0,72.0,59.0,94.0,48.0,22.0,94.0,94.0,75.0,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,110500000.0,565000.0,226500000.0,159,170.18,0,1,unique,1,1
1,33,Portugal,94,94,Juventus,2228,5.0,4.0,5.0,ST,"Jul 10, 2018",NaN,2022,92.500000,92.500000,92.500000,90.500000,91.500000,91.500000,91.500000,90.500000,89.500000,89.500000,89.500000,89.500000,82.500000,82.500000,82.500000,89.500000,66.500000,62.500000,62.500000,62.500000,66.500000,62.500000,54.500000,54.500000,54.500000,62.500000,84.0,94.0,89.0,81.0,87.0,88.0,81.0,76.0,77.0,94.0,89.0,91.0,87.0,96.0,70.0,95.0,95.0,88.0,79.0,93.0,63.0,29.0,95.0,82.0,85.0,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,77000000.0,405000.0,127100000.0,183,187.96,1,1,unique,2,0
2,26,Brazil,92,93,Paris Saint-Germain,2143,5.0,5.0,5.0,LW,"Aug 3, 2017",NaN,2022,85.500000,85.500000,85.500000,90.500000,90.500000,90.500000,90.500000,90.500000,90.500000,90.500000,90.500000,89.500000,82.500000,82.500000,82.500000,89.500000,66.500000,61.500000,61.500000,61.500000,66.500000,61.500000,48.500000,48.500000,48.500000,61.500000,79.0,87.0,62.0,84.0,84.0,96.0,88.0,87.0,78.0,95.0,94.0,90.0,96.0,94.0,84.0,80.0,61.0,81.0,49.0,82.0,56.0,36.0,89.0,87.0,81.0,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,118500000.0,290000.0,228100000.0,150,175.26,1,1,unique,2,1
3,27,Spain,91,93,Manchester United,1471,4.0,3.0,1.0,GK,"Jul 1, 2011",NaN,2020,69.266667,69.266667,69.266667,70.566667,70.466667,70.466667,70.466667,70.566667,70.966667,70.966667,70.966667,71.266667,70.466667,70.466667,70.466667,71.266667,68.733333,68.733333,68.733333,68.733333,68.733333,67.733333,66.633333,66.633333,66.633333,67.733333,17.0,13.0,21.0,50.0,13.0,18.0,21.0,19.0,51.0,42.0,57.0,58.0,60.0,90.0,43.0,31.0,67.0,43.0,64.0,12.0,38.0,30.0,12.0,68.0,40.0,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,72000000.0,260000.0,138600000.0,168,193.04,1,1,Lean,1,1
4,27,Belgium,91,92,Manchester City,2281,4.0,5.0,4.0,RCM,"Aug 30, 2015",NaN,2023,83.500000,83.500000,83.500000,88.500000,88.500000,88.500000,88.500000,88.500000,89.500000,89.500000,89.500000,89.500000,88.500000,88.500000,88.500000,89.500000,78.500000,78.500000,78.500000,78.500000,78.500000,74.500000,67.500000,67.500000,67.500000,74.500000,93.0,82.0,55.0,92.0,82.0,86.0,85.0,83.0,91.0,91.0,78.0,76.0,79.0,91.0,77.0,91.0,63.0,90.0,75.0,91.0,76.0,61.0,87.0,94.0,79.0,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,102000000.0,355000.0,196400000.0,154,180.34,1,1,Normal,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17913,19,England,47,65,Crewe Alexandra,1307,1.0,2.0,2.0,CM,"May 3, 2017",NaN,2019,43.000000,43.000000,43.000000,45.000000,45.000000,45.000000,45.000000,45.000000,46.000000,46.000000,46.00

## Atributos en el equipo

Comprobamos la gente que tiene la fecha de entrada nula 

In [32]:
print(len(dfc.loc[dfc['Joined'].isna()]))

1264


Les aplicaremos el día de entrada como el día que entró el último jugador + 1, es decir, serán los últimos fichajes. Así podremos decir que son los quew llevan menos tiempo en el equipo.

In [33]:
maxdatedf = pd.DataFrame()
maxdatedf['Joined'] = df['Joined'].dropna()
maxdatedf['joined'] = maxdatedf['Joined'].apply(lambda x: \
            datetime.datetime.strptime(x, '%b %d, %Y'))
lasttime = (max(maxdatedf['joined']) 
 + datetime.timedelta(days=1)).strftime('%b %d, %Y')

In [34]:
copy = df['Joined'].copy()
copy.loc[df['Joined'].isna()] = lasttime
df['Joined'] = copy

Calculamos el número de días que lleva en el equipo tomando en cuenta el último día como máximo.

In [35]:
df['joined'] = df['Joined'].apply(lambda x: \
            datetime.datetime.strptime(x, '%b %d, %Y'))
maxtime = max(maxdatedf['joined']) + datetime.timedelta(days=1)
dfc = df.drop(['Joined'], axis=1)
dfc['timein'] = dfc['joined'].apply(lambda x: \
                             (maxtime - x).days)
df = dfc

El tipo de contrato válido esté en un tipo incorrecto

In [36]:
df['Contract Valid Until'].dtypes

dtype('O')

Lo pasamos a numérico

In [37]:
df['Contract Valid Until'] = pd.to_numeric(df['Contract Valid Until'], \
                                           errors='coerce')

In [38]:
df['Contract Valid Until'].dtypes

dtype('float64')

Miramos si hay nulls

In [39]:
df.loc[df['Contract Valid Until'].isnull()]

,Age,Nationality,Overall,Potential,Club,Special,International Reputation,Weak Foot,Skill Moves,Position,Loaned From,Contract Valid Until,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,value,wage,rclause,weight,height,rfoot,rf,Body Type,wrh,wrl,joined,timein
28,26,Colombia,88,89,FC Bayern München,2171,4.0,3.0,4.0,LAM,Real Madrid,NaN,81.5,81.5,81.5,85.5,84.5,84.5,84.5,85.5,86.5,86.5,86.5,84.5,82.5,82.5,82.5,84.5,70.5,69.5,69.5,69.5,70.5,66.5,59.5,59.5,59.5,66.5,90.0,83.0,62.0,89.0,90.0,85.0,89.0,86.0,83.0,90.0,73.0,67.0,83.0,85.0,76.0,86.0,54.0,70.0,68.0,92.0,64.0,55.0,80.0,89.0,81.0,87.0,52.0,41.0,44.0,15.0,15.0,15.0,5.0,14.0,69500000.0,315000.0,3.698279e+07,172,180.34,0,1,Normal,1,1,2018-12-21,0
38,30,Argentina,88,88,Milan,1965,4.0,4.0,3.0,LS,Juventus,NaN,86.5,86.5,86.5,82.5,84.5,84.5,84.5,82.5,81.5,81.5,81.5,79.5,72.5,72.5,72.5,79.5,56.5,54.5,54.5,54.5,56.5,52.5,49.5,49.5,49.5,52.5,68.0,92.0,80.0,75.0,90.0,84.0,74.0,62.0,59.0,85.0,73.0,73.0,75.0,86.0,69.0,86.0,79.0,70.0,85.0,80.0,50.0,20.0,92.0,74.0,70.0,86.0,35.0,22.0,18.0,5.0,12.0,7.0,5.0,10.0,57000000.0,245000.0,2.435367e+07,196,185.42,1,1,Normal,2,1,2018-12-21,0
91,29,Brazil,85,85,Guangzhou Evergrande Taobao FC,2245,3.0,3.0,3.0,LDM,FC Barcelona,NaN,82.5,82.5,82.5,80.5,82.5,82.5,82.5,80.5,82.5,82.5,82.5,80.5,83.5,83.5,83.5,80.5,79.5,82.5,82.5,82.5,79.5,79.5,81.5,81.5,81.5,79.5,64.0,81.0,85.0,84.0,75.0,82.0,66.0,72.0,80.0,84.0,73.0,73.0,69.0,84.0,66.0,83.0,90.0,92.0,81.0,80.0,81.0,81.0,84.0,78.0,71.0,80.0,75.0,81.0,73.0,16.0,16.0,11.0,6.0,8.0,37000000.0,235000.0,3.584538e+06,157,182.88,1,1,Lean,2,2,2018-12-21,0
166,24,Brazil,83,90,Guangzhou Evergrande Taobao FC,2142,2.0,4.0,4.0,CAM,SL Benfica,NaN,81.0,81.0,81.0,82.0,82.0,82.0,82.0,82.0,82.0,82.0,82.0,81.0,80.0,80.0,80.0,81.0,70.0,70.0,70.0,70.0,70.0,68.0,65.0,65.0,65.0,68.0,78.0,80.0,77.0,81.0,76.0,82.0,86.0,87.0,79.0,84.0,77.0,76.0,76.0,79.0,65.0,84.0,58.0,76.0,72.0,88.0,67.0,58.0,86.0,81.0,71.0,74.0,55.0,62.0,42.0,13.0,11.0,13.0,12.0,10.0,36500000.0,18000.0,3.584538e+06,176,190.50,0,0,Normal,2,1,2018-12-21,0
176,24,Croatia,83,89,Chelsea,2115,3.0,4.0,4.0,LCM,Real Madrid,NaN,72.5,72.5,72.5,80.5,79.5,79.5,79.5,80.5,82.5,82.5,82.5,81.5,83.5,83.5,83.5,81.5,77.5,79.5,79.5,79.5,77.5,75.5,70.5,70.5,70.5,75.5,73.0,62.0,42.0,87.0,76.0,88.0,78.0,68.0,82.0,89.0,84.0,74.0,81.0,70.0,85.0,79.0,63.0,82.0,65.0,75.0,77.0,78.0,69.0,88.0,59.0,72.0,69.0,75.0,63.0,9.0,5.0,6.0,7.0,7.0,35000000.0,165000.0,3.415044e+07,172,175.26,1,1,Normal,2,2,2018-12-21,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17689,21,England,51,57,Hamilton Academical FC,1277,1.0,2.0,2.0,ST,Norwich City,NaN,50.0,50.0,50.0,50.0,50.0,50.0,50.0,50.0,48.0,48.0,48.0,48.0,41.0,41.0,41.0,48.0,35.0,32.0,32.0,32.0,35.0,34.0,31.0,31.0,31.0,34.0,34.0,53.0,44.0,38.0,40.0,50.0,39.0,26.0,33.0,48.0,64.0,74.0,56.0,49.0,65.0,42.0,62.0,48.0,60.0,43.0,35.0,16.0,47.0,45.0,62.0,49.0,14.0,13.0,13.0,12.0,13.0,16.0,9.0,14.0,50000.0,3000.0,3.935417e+05,165,180.34,1,0,Lean,1,1,2018-12-21,0
17690,21,China PR,51,60,Guizhou Hengfeng FC,1459,1.0,2.0,2.0,CM,Tianjin Quanjian FC,NaN,50.0,50.0,50.0,51.0,51.0,51.0,51.0,51.0,51.0,51.0,51.0,52.0,51.0,51.0,51.0,52.0,51.0,51.0,51.0,51.0,51.0,51.0,50.0,50.0,50.0,51.0,36.0,30.0,48.0,57.0,32.0,45.0,37.0,38.0,55.0,50.0,71.0,76.0,75.0,55.0,58.0,55.0,60.0,62.0,61.0,33.0,58.0,41.0,5

Parece que coincide con las nuevas adquisiciones, así que lo que haremos es poner la media de duración de cada uno de los jugadores. 

Primero, calculamos los tiempos medios de contratos de los jugadores de cada equipo

In [40]:
contratodf = pd.DataFrame()
contratodf['CVU'] = df['Contract Valid Until'].copy()
contratodf['joined'] = df['joined'].copy()
contratodf['Club'] = df['Club'].copy()
contratodf = contratodf.dropna()
contratodf['CVU']
contratodf['validity'] = contratodf.apply(lambda row: \
           (datetime.datetime(int(row['CVU']), 1, 1) - row['joined']).days , 
                                          axis = 1)

meanclub = {}
for i in contratodf['Club'].unique():
    meanclub[i] = contratodf.loc[contratodf['Club'] == i]['validity'].mean()

Los aplicamos a todos los jugadores si tienen nan, sino, les ponemos sui tiempo de validez

In [41]:
df['validity'] = df.apply(lambda row: \
            meanclub[row['Club']] if pd.isnull(row['Contract Valid Until'])
            else (datetime.datetime(int(row['Contract Valid Until']), 1, 1) - \
                  row['joined']).days, axis = 1)
df['validity']

0        6028.0
1        1271.0
2        1612.0
3        3106.0
4        2681.0
          ...  
17913     608.0
17914     653.0
17915    1280.0
17916     252.0
17917      63.0
Name: validity, Length: 17918, dtype: float64

Y eliminamos las columnas innecesarias porque son fechas, que no son procesables.

In [42]:
df = df.drop(['Contract Valid Until', 'joined'], axis = 1)

## Factorización

Factorizamos todas las varibales categóricas:

In [43]:
factors = [['Nationality','nat'],['Position','pos'],['Club','clb'], \
           ['Body Type', 'body'],['Loaned From', 'lf']]
for i in factors:
    df.dropna(subset = [i[0]])
    data = df.pop(i[0])
    df = pd.concat([df.reset_index(drop=True), \
            pd.get_dummies(data, prefix=i[1]).reset_index(drop=True)], \
            axis=1, sort=False)

In [44]:
len(df.columns)

1264

## Partición y Normalización

Sacamos el valor a predecir:

In [45]:
dfc = df.copy()
val = dfc.pop("value")

Hacemos un split de 1/3 del dataset usando el 1/3 para probar los resultados con los 2/3 del dataset que se usarán para hacer el training

In [46]:
X_train, X_test, y_train, y_test = train_test_split(dfc, val, \
                                                    test_size=0.33, \
                                                    random_state=42)

Normalizamos todas las variables numéricas excepto value (lo que hay que predecir) y las variables que ya están entre 0 y 1:

In [47]:
trcopy = X_train.copy()
tscopy = X_test.copy()
for i in X_train.columns:
    if i != 'value' and X_train[i].min() != 0 and X_train[i].max() != 1:
        trcopy[i] = (X_train[i]-X_train[i].min())/ \
                        (X_train[i].max()-X_train[i].min())
        tscopy[i] = (X_test[i]-X_test[i].min())/(X_test[i].max()-X_test[i].min())
X_train = trcopy
X_test = tscopy

## Predicción

Entrenamos el modelo

In [48]:
reg = linear_model.LinearRegression().fit(X_train, y_train)

In [49]:
preds = reg.predict(X_test)

In [50]:
preds[0]

148630.3125

In [51]:
y_test[0]

110500000.0

Obtenemos el valor de error de R²

In [52]:
r2_score(y_true = y_test, y_pred = preds)

-3073081597.9764953

Parece que es un número extremadamente negativo. Esto puede ser debido a la alta dimensionalidad de nuestro dataframe. Para eso aplicamos PCA, método que reduce la dimensionalidad mediante al unión de columnas.

Aplicamos PCA, pasamos a 1000 columnas. Entrenamos con X_train, aplicamos sobre X_test.

In [53]:
pca=PCA(n_components=1000)
pca_train=pd.DataFrame(pca.fit_transform(X_train))
pca_test=pd.DataFrame(pca.transform(X_test))

Volvemos a entrenar el modelo

In [54]:
reg = linear_model.LinearRegression().fit(pca_train, y_train)

In [55]:
preds = reg.predict(pca_test)

In [56]:
for i in list(preds[0:10]):
    print(int(i))

-133446
1100634
84751
1333784
6296705
117272
13517743
1646554
644596
-237148


In [57]:
for i in list(y_test[0:10]):
    print(int(i))

200000
800000
425000
1300000
3700000
375000
13500000
950000
600000
120000


Obtenemos el valor de error de R²

In [58]:
r2_score(y_true = y_test, y_pred = preds)

0.9730387259619596

Como podemos ver, hay predicciones negativas, pero el valor de R² es muy cercano a 1. Esta es la mejor forma de predecir los precios de los jugadores hayada.